# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import keras
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

import logging

logging.getLogger().setLevel(logging.INFO)

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
time_steps_per_day = 6
batch_size = 50

sim_dirs = [
    ('NYC_Heat_Test', [
        'NYC_summer_2000_01p',
        # 'NYC_summer_2010_99p',
        # 'NYC_summer_2015_50p',
        # 'NYC_summer_2017_25p',
        # 'NYC_summer_2018_75p'
    ]),
    ('PHX_Heat_Test', [
        'PHX_summer_2008_25p',
        # 'PHX_summer_2009_50p',
        # 'PHX_summer_2011_99p',
        # 'PHX_summer_2015_75p',
        # 'PHX_summer_2020_01p'
    ])
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + '/' + subdir)

print(sim_names)
client = storage.Client(project="climateiq")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['NYC_Heat_Test/NYC_summer_2000_01p', 'NYC_Heat_Test/NYC_summer_2010_99p', 'NYC_Heat_Test/NYC_summer_2015_50p', 'NYC_Heat_Test/NYC_summer_2017_25p', 'NYC_Heat_Test/NYC_summer_2018_75p', 'PHX_Heat_Test/PHX_summer_2008_25p', 'PHX_Heat_Test/PHX_summer_2009_50p', 'PHX_Heat_Test/PHX_summer_2011_99p', 'PHX_Heat_Test/PHX_summer_2015_75p', 'PHX_Heat_Test/PHX_summer_2020_01p']


In [7]:
# Create the training dataset using create_atmo_dataset with sim_names
train_frac = 0.8

train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    hash_range=(0.0, train_frac),
).batch(batch_size=batch_size)

val_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    hash_range=(train_frac, 1.0),
).batch(batch_size=batch_size)

INFO:root:sim_name_dates [('PHX_Heat_Test/PHX_summer_2008_25p', '2008-07-25'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-07-14'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-08-08'), ('PHX_Heat_Test/PHX_summer_2009_50p', '2009-08-16'), ('NYC_Heat_Test/NYC_summer_2018_75p', '2018-07-04'), ('NYC_Heat_Test/NYC_summer_2018_75p', '2018-07-20'), ('PHX_Heat_Test/PHX_summer_2008_25p', '2008-06-23'), ('PHX_Heat_Test/PHX_summer_2011_99p', '2011-06-07'), ('PHX_Heat_Test/PHX_summer_2015_75p', '2015-07-09'), ('NYC_Heat_Test/NYC_summer_2015_50p', '2015-07-15'), ('NYC_Heat_Test/NYC_summer_2015_50p', '2015-06-17'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-07-26'), ('PHX_Heat_Test/PHX_summer_2008_25p', '2008-07-12'), ('PHX_Heat_Test/PHX_summer_2008_25p', '2008-08-24'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-06-21'), ('PHX_Heat_Test/PHX_summer_2020_01p', '2020-07-13'), ('PHX_Heat_Test/PHX_summer_2009_50p', '2009-07-19'), ('NYC_Heat_Test/NYC_summer_2015_50p', '2015-06-12'), ('NYC_Heat_Test/NYC_

In [10]:
num_samples = 0
for batch in train_ds:
    num_samples += batch[0]['spatiotemporal'].shape[0]
print("Number of samples:", num_samples)

INFO:root:load_day (2008-07-25, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2018-07-20, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2011-06-07, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2015-07-15, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2018-08-23, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2015-08-28, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2009-06-25, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2017-07-24, NYC_Heat_Test/NYC_summer_2017_25p)
INFO:root:load_day (2015-08-31, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2010-06-30, NYC_Heat_Test/NYC_summer_2010_99p)
INFO:root:load_day (2008-08-23, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2008-07-06, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2011-08-24, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2008-06-14, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2015-06-07, PHX_Heat_Test/PHX_summer_2015_

Number of samples: 158


In [11]:
num_samples = 0
for batch in val_ds:
    num_samples += batch[0]['spatiotemporal'].shape[0]
print("Number of samples:", num_samples)

INFO:root:load_day (2015-06-18, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2020-08-09, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2015-08-09, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2008-07-02, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2011-05-24, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2009-08-30, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2011-06-24, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2015-05-31, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2009-08-08, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2008-08-19, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2009-07-27, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2015-07-01, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2015-08-14, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2018-07-23, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2010-08-28, NYC_Heat_Test/NYC_summer_2010_

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (4, 200, 200, 12) where an element of shape (6, 200, 200, 12) was expected.
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (4, 200, 200, 12) where an element of shape (6, 200, 200, 12) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext] name: 

INFO:root:load_day (2009-08-23, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2008-07-03, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2015-08-06, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2008-07-29, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2009-08-16, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2015-07-05, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2020-06-09, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2015-08-11, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2000-08-25, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2011-08-09, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2020-08-15, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2000-06-10, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2008-08-25, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2010-08-17, NYC_Heat_Test/NYC_summer_2010_99p)
INFO:root:load_day (2015-07-13, PHX_Heat_Test/PHX_summer_2015_

In [12]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)

INFO:root:load_day (2017-05-24, NYC_Heat_Test/NYC_summer_2017_25p)
INFO:root:load_day (2015-08-21, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2009-07-29, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2020-08-13, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2008-05-29, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2008-07-16, PHX_Heat_Test/PHX_summer_2008_25p)


In [13]:
# Train the model
tb_callback = keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(train_ds, val_ds, epochs=1, callbacks=[tb_callback])

Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)
Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)


INFO:root:load_day (2000-08-15, NYC_Heat_Test/NYC_summer_2000_01p)
2024-12-14 00:19:22.290293: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inatmo_conv_lstm_1/conv_lstm/conv_lstm2d_1/while/body/_1/atmo_conv_lstm_1/conv_lstm/conv_lstm2d_1/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Output shape: TensorShape([None, 8, 50, 50, 256])


INFO:root:load_day (2008-07-25, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2009-05-29, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2018-07-20, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2011-05-28, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2011-06-07, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2009-07-26, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2011-08-03, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2015-07-15, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2015-08-10, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2018-08-23, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2015-07-17, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2015-08-28, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2011-08-02, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2009-06-25, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2011-07-12, PHX_Heat_Test/PHX_summer_2011_

      1/Unknown - 110s 110s/step - loss: 18692.8574 - mean_absolute_error: 69.4132 - root_mean_squared_error: 136.7218Output shape: TensorShape([None, 8, 50, 50, 256])


...

INFO:root:load_day (2009-06-29, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2008-07-18, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2009-07-20, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2009-07-28, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2015-05-28, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2017-06-03, NYC_Heat_Test/NYC_summer_2017_25p)
INFO:root:load_day (2015-08-25, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2020-08-07, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2015-05-29, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2015-07-16, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2000-07-15, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2015-07-30, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2015-06-04, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2020-06-18, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2010-07-14, NYC_Heat_Test/NYC_summer_2010_

Output shape: TensorShape([None, 8, 50, 50, 256])


INFO:root:load_day (2008-07-22, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2011-05-30, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2018-05-25, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2018-06-18, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2018-07-09, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2018-06-27, NYC_Heat_Test/NYC_summer_2018_75p)
INFO:root:load_day (2015-08-29, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2020-06-25, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2010-06-08, NYC_Heat_Test/NYC_summer_2010_99p)
INFO:root:load_day (2015-07-27, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2008-07-05, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2000-06-30, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2010-06-29, NYC_Heat_Test/NYC_summer_2010_99p)
INFO:root:load_day (2009-07-03, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2020-06-10, PHX_Heat_Test/PHX_summer_2020_

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: `generator` yielded an element of shape (4, 200, 200, 12) where an element of shape (6, 200, 200, 12) was expected.
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (4, 200, 200, 12) where an element o

INFO:root:load_day (2009-08-23, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2008-07-03, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2015-08-06, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2008-07-29, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2009-08-16, PHX_Heat_Test/PHX_summer_2009_50p)
INFO:root:load_day (2015-07-05, PHX_Heat_Test/PHX_summer_2015_75p)
INFO:root:load_day (2020-06-09, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2015-08-11, NYC_Heat_Test/NYC_summer_2015_50p)
INFO:root:load_day (2000-08-25, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2011-08-09, PHX_Heat_Test/PHX_summer_2011_99p)
INFO:root:load_day (2020-08-15, PHX_Heat_Test/PHX_summer_2020_01p)
INFO:root:load_day (2000-06-10, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2008-08-25, PHX_Heat_Test/PHX_summer_2008_25p)
INFO:root:load_day (2010-08-17, NYC_Heat_Test/NYC_summer_2010_99p)
INFO:root:load_day (2015-07-13, PHX_Heat_Test/PHX_summer_2015_

In [5]:
model._model.summary()

Model: "atmo_conv_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  488       
                                                                 
 spatial_cnn (Sequential)    (None, 50, 50, 128)       252992    
                                                                 
 spatiotemporal_cnn (Sequen  (None, None, 50, 50, 64   30480     
 tial)                       )                                   
                                                                 
 conv_lstm (Sequential)      (None, None, 50, 50, 51   45877248  
                             2)                                  
                                                                 
 t2_output_cnn (Sequential)  (None, None, 200, 200,    69729     
                             1)                                  
                                                    

In [6]:
# Test calling the model on some training data
inputs, labels = next(iter(train_dataset))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)

NameError: name 'train_dataset' is not defined